# PDF Semantic Analysis

In [1]:
title = "From Few to Many: Self-Improving Many-Shot Reasoners Through Iterative Optimization and Generation"

## Metadata Info

### Paper Metadata

In [2]:
import sys
import os

# 获取当前脚本所在目录的父目录 (即 my_project)
parent_dir = os.path.dirname(os.getcwd())

# 将父目录添加到 sys.path
sys.path.append(parent_dir)

In [3]:
from apis.arxiv_tool import ArxivKit
from apis.semanticscholar_tool import SemanticScholarKit

In [ ]:
arxiv = ArxivKit()
arxiv_metadata_lst = arxiv.retrieve_metadata_by_paper(query_term=title, max_cnt=3)

In [ ]:
arxiv_metadata_lst[0].keys()

In [ ]:
ss = SemanticScholarKit()
ss_metadata_lst = ss.search_paper_by_keywords(query=title, limit=3)

In [ ]:
ss_metadata_lst[0].keys()

In [ ]:
paper_ss_id = ss_metadata_lst[0].get('paperId')

### Citations & References

In [ ]:
reference_metadata_lst = ss.get_semanticscholar_references(paper_id='603bb1eb011147bed0766c0b0c0bf4bb83ef4b9a', limit=100)

In [ ]:
citedby_metadata_lst = ss.get_semanticscholar_citedby(paper_id='603bb1eb011147bed0766c0b0c0bf4bb83ef4b9a', limit=100)

In [ ]:
citedby_metadata_lst[0].__dict__.get('_data').keys()

### Recommended Papers

In [ ]:
recommended_papers_lst = ss.find_recommendations(positive_paper_ids=[paper_ss_id], limit=100)

In [ ]:
type(recommended_papers_lst[0])

## Generate Categories, Keywords and Tags

In [ ]:
    def upload_file(api_key, file_path):
        # filename = os.path.splitext(os.path.basename(file_path))[0].lower()
        myfile = genai.upload_file(path=file_path)
        return myfile.name  # "files/*"

    def gemini_remove_file(api_key, filename):
        genai.configure(api_key=api_key)
        if not filename.startswith("files/"):
            filename = f"files/{filename}"

        all_files = genai.list_files()
        if filename in all_files:
            genai.delete_file(filename)

In [ ]:
    def gemini_llm_w_files(api_key, model_name, prompt, filenames):
        genai.configure(api_key=api_key)
        all_files = genai.list_files()
        valid_filenames = []
        for filename in filenames:
            if not filename.startswith("files/"):
                filename = f"files/{filename}"
            if filename in all_files:
                valid_filenames.append(filename)
        model = genai.GenerativeModel(api_key=api_key, model_name=model_name)
        response = model.generate_content([prompt] + valid_filenames)
        return (response.text)


    def gemini_llm(self, api_key, model_name, prompt, if_json=False, response_class=None, file_paths=None, image_paths=None):
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(model_name=model_name)
        
        # set up parameters
        # output format
        if if_json:
            response_mime_type = "application/json"
        else:
            response_mime_type = "text/plain"
        # output schema
        if response_class:
            response_scheme = list[response_class]
        else:
            response_scheme = None

        # generation with files
        if file_paths:
            # check if local file or file that uploaded
            filenms = []
            for file in file_paths:
                myfile = genai.upload_file(path=file)
                filenms.append(myfile.name)
            contents = [prompt] + filenms
        
        # generation with images
        if image_paths:
            images= []
            # check if local file or file that uploaded
            for image in image_paths:
                images.append(img.open(image))
            contents = [prompt] + images

        model_config = genai.GenerationConfig(
                response_mime_type=response_mime_type, response_schema=response_scheme
            )

        result = model.generate_content(
            contents=contents,
            generation_config=model_config,
        )
        print(result) 